# Assignment: Segmenting and Clustering Neighborhoods in Sandton
Notebook to scrape and understand Neighbourhood information

### Install and Import Packages

In [ ]:
# install scraping package
#!conda install -c conda-forge beautifulsoup4

# library import Section 
import numpy as np
import pandas as pd
from pandas import DataFrame

import requests
from bs4 import BeautifulSoup
# from tabulate import tabulate


import requests # library to handle requests
import random # library for random number generation

# !conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library


Solving environment: | 

### Scrape the input data
#### Build the Scaping Function

In [2]:
def extracturl(url):
    import requests

    header = {
      "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
      "X-Requested-With": "XMLHttpRequest"
    }

    r = requests.get(url, headers=header)

    dfs = pd.read_html(r.text)

    tables = pd.read_html(r.text)
    df = tables[3]
    print('Inital shape: ' + str(df.shape))
    df
    return(df)

#### Define the extraction URLs

In [3]:
urlsandton = "https://census2011.adrianfrith.com/place/798013"
urlu='https://census2011.adrianfrith.com/place/599054'
url='https://census2011.adrianfrith.com/place/199041'

#### Extract the URLs

In [4]:
df_ct=extracturl(url)
df_ct['Main Place']='Cape Town'
df_ct.head(5)

Inital shape: (122, 3)


,Name,Population,Area (km²),Main Place
0,Airlie,312,0.26,Cape Town
1,Alphen,754,2.32,Cape Town
2,Bakoven,2209,1.35,Cape Town
3,Bantry Bay,820,0.38,Cape Town
4,Barbarossa,927,0.57,Cape Town


In [5]:
df_s=extracturl(urlsandton)
df_s['Main Place']='Sandton'
df_s.head(5)

Inital shape: (62, 3)


,Name,Population,Area (km²),Main Place
0,Atholl,4153,2.24,Sandton
1,Barlow Park,234,0.51,Sandton
2,Beverley,3828,2.70,Sandton
3,Bramley North,553,0.30,Sandton
4,Bramley Park,1161,0.54,Sandton


In [6]:
df_u=extracturl(urlu)
df_u['Main Place']='Durban'
df_u.head(5)

Inital shape: (71, 3)


,Name,Population,Area (km²),Main Place
0,Athlone,2851,1.38,Durban
1,Athlone Park,3818,4.35,Durban
2,Avoca,7568,2.53,Durban
3,Bayhead,365,12.07,Durban
4,Beachwood Mangroves,201,2.37,Durban


In [7]:
df_u['address']=df_u['Name']+','+df_u['Main Place']
print(' SubPlaces Extract')
df_s['address']=df_s['Name']+','+df_s['Main Place']
df_ct['address']=df_ct['Name']+','+df_ct['Main Place']

 SubPlaces Extract


In [8]:
# dfu['address']=dfu['Name']+','+dfu['Main Place']
# Check the final dataframe
df_u.head(5)

,Name,Population,Area (km²),Main Place,address
0,Athlone,2851,1.38,Durban,"Athlone,Durban"
1,Athlone Park,3818,4.35,Durban,"Athlone Park,Durban"
2,Avoca,7568,2.53,Durban,"Avoca,Durban"
3,Bayhead,365,12.07,Durban,"Bayhead,Durban"
4,Beachwood Mangroves,201,2.37,Durban,"Beachwood Mangroves,Durban"


## Setup foursquare API

In [9]:
CLIENT_ID = 'D20PTFA5ZWZHCWP0UD4OISYG5GACXPGCFWH0WBO42RGINHX3' # your Foursquare ID
CLIENT_SECRET = 'JYCOZZ1FYSL5YIU0PL1UDHM25DCIF4P41UG0YQLFKGVLQK2O' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: D20PTFA5ZWZHCWP0UD4OISYG5GACXPGCFWH0WBO42RGINHX3
CLIENT_SECRET:JYCOZZ1FYSL5YIU0PL1UDHM25DCIF4P41UG0YQLFKGVLQK2O


## Create Geolocation Functions

In [10]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import time

def locate(address):

#     address = 'Atholl, Sandton'
    geolocator = Nominatim(user_agent="explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return(latitude,longitude,location )

In [11]:
import time

def get_coordinates(address, timeout=5):
    geolocator = Nominatim(user_agent="explorer")
    location = geolocator.geocode(address, timeout=timeout)
    time.sleep(1)
    if not location:
        return None, None
    return str(location.latitude), str(location.longitude)

In [12]:
## Test GPS Functions

print(get_coordinates('Sandton,Bramley Park'))
print(locate('Sandton,Lonehill'))

('-26.1247222', '28.0788889')
(-26.0097222, 28.0261111, Location(Lonehill, Johannesburg, City of Johannesburg Metropolitan Municipality, Gauteng, 2001, South Africa, (-26.0097222, 28.0261111, 0.0)))


In [13]:
## Lookup Addresses
df_u['Coords'] = df_u['address'].apply(get_coordinates)
df_s['Coords'] = df_s['address'].apply(get_coordinates)
df_ct['Coords'] = df_ct['address'].apply(get_coordinates)
df_ct_xy=df_ct
df_s_xy=df_s
df_u_xy=df_u
print('Lookup Complete')

Lookup Complete


## Split Coords into Lat and Long
### Define the function

In [14]:
def splitxy(dataf):
    d = dataf["Coords"].astype(str).str.split(',',expand=True) 

    d[0]=d[0].str.replace("(", '')
    d[0]=d[0].str.replace("'", '')
    d[1]=d[1].apply(str).str.replace(")", '')
    d[1]=d[1].str.replace("'", '')


    dataf["Latitude"]=d[0]
    dataf["Longitude"]=d[1]

    
    return dataf

In [15]:
splitxy(df_u_xy)
splitxy(df_ct_xy)
splitxy(df_s_xy)
print('Split XY done')

Split XY done


## Remove None Fields


In [16]:
df_s_xy=df_s_xy[df_s_xy['Latitude'] != 'None']
df_ct_xy=df_ct_xy[df_ct_xy['Latitude'] != 'None']
df_u_xy=df_u_xy[df_u_xy['Latitude'] != 'None']
print('done removing None Fields')

done removing None Fields


In [17]:
# Check dataframe
df_ct_xy.head()

,Name,Population,Area (km²),Main Place,address,Coords,Latitude,Longitude
0,Airlie,312,0.26,Cape Town,"Airlie,Cape Town","(-34.041163, 18.435566)",-34.041163,18.435566
1,Alphen,754,2.32,Cape Town,"Alphen,Cape Town","(-34.008307, 18.43920808836982)",-34.008307,18.43920808836982
2,Bakoven,2209,1.35,Cape Town,"Bakoven,Cape Town","(-33.96, 18.3827778)",-33.96,18.3827778
3,Bantry Bay,820,0.38,Cape Town,"Bantry Bay,Cape Town","(-33.9281512, 18.3789695)",-33.9281512,18.3789695
5,Bel Ombre,1045,2.02,Cape Town,"Bel Ombre,Cape Town","(-34.0000223, 18.4074868)",-34.0000223,18.4074868


## Folium map view of Sandton

In [18]:
# create map of Sandton using latitude and longitude values
map_sandton = folium.Map(location=[-26.064634, 28.034603], zoom_start=12)

# add markers to map
for lat, lng, Name, Population in zip(df_s_xy['Latitude'], df_s_xy['Longitude'], df_s_xy['Name'], df_s_xy['Population']):
    label = '{}, {}'.format(Name, Population)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lng)],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_sandton)  
    
map_sandton

In [19]:
# create map of Sandton using latitude and longitude values
map_ct = folium.Map(location=[-34.041163, 18.435566], zoom_start=11)

# add markers to map
for lat, lng, Name, Population in zip(df_ct_xy['Latitude'], df_ct_xy['Longitude'], df_ct_xy['Name'], df_ct_xy['Population']):
    label = '{}, {}'.format(Name, Population)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lng)],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#e35e46',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ct)  
    
map_ct

In [20]:
# create map of Durban using latitude and longitude values
map_umh = folium.Map(location=[-29.8587, 31.0218], zoom_start=11)

# add markers to map
for lat, lng, Name, Population in zip(df_u_xy['Latitude'], df_u_xy['Longitude'], df_u_xy['Name'], df_u_xy['Population']):
    label = '{}, {}'.format(Name, Population)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lng)],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#46e34e',
        fill_opacity=0.7,
        parse_html=False).add_to(map_umh)  
    
map_umh

##   Explore Neighborhoods

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
## Sandton Neighbourhood
sandton_venues = getNearbyVenues(names=df_s_xy['Name'],
                                   latitudes=df_s_xy['Latitude'],
                                   longitudes=df_s_xy['Longitude']
                                  )


Atholl


KeyError: 'groups'

In [26]:
## Durban Neighbourhood
dbn_venues = getNearbyVenues(names=df_u_xy['Name'],
                                   latitudes=df_u_xy['Latitude'],
                                   longitudes=df_u_xy['Longitude']
                                  )


Athlone


KeyError: 'groups'

In [27]:
## Cape Town Neighbourhood

ct_venues = getNearbyVenues(names=df_ct_xy['Name'],
                                   latitudes=df_ct_xy['Latitude'],
                                   longitudes=df_ct_xy['Longitude']
                                  )

Airlie


KeyError: 'groups'

In [ ]:
sandton_venues.groupby(['Venue Category'])['Venue Category'].count()
ct_venues.groupby(['Venue Category'])['Venue Category'].count()
dbn_venues.groupby(['Venue Category'])['Venue Category'].count()
print('done')

In [ ]:
%matplotlib inline
dbn_venues=dbn_venues[dbn_venues['Venue Category'].str.contains("Restaurant")]
dbn_venues=dbn_venues[dbn_venues['Venue Category'] != 'Restaurant']
dbn_venues=dbn_venues[dbn_venues['Venue Category'] != 'Fastfood Restaurant']
dbn_venues=dbn_venues[dbn_venues['Venue Category'] != 'Fast Food Restaurant']

ax = dbn_venues['Venue Category'].value_counts().plot(kind='bar',sharex=True,sharey=True,title='Restaurant Types in Durban')

In [ ]:
%matplotlib inline
sandton_venues=sandton_venues[sandton_venues['Venue Category'].str.contains("Restaurant")]
sandton_venues=sandton_venues[sandton_venues['Venue Category'] != 'Restaurant']
sandton_venues=sandton_venues[sandton_venues['Venue Category'] != 'Fastfood Restaurant']
sandton_venues=sandton_venues[sandton_venues['Venue Category'] != 'Fast Food Restaurant']
ax=sandton_venues['Venue Category'].value_counts().plot(kind='bar',sharex=True,sharey=True,title='Restaurant Types in Sandton, Johannesburg')

In [ ]:
%matplotlib inline
ct_venues=ct_venues[ct_venues['Venue Category'].str.contains("Restaurant")]
ct_venues=ct_venues[ct_venues['Venue Category'] != 'Restaurant']
ct_venues=ct_venues[ct_venues['Venue Category'] != 'Fastfood Restaurant']
ct_venues=ct_venues[ct_venues['Venue Category'] != 'Fast Food Restaurant']
ax=ct_venues['Venue Category'].value_counts().plot(kind='bar',sharex=True,sharey=True,title='Restaurant Types in Cape Town')

## Clustering

In [ ]:
# one hot encoding
sandton_venues_onehot = pd.get_dummies(sandton_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sandton_venues_onehot['Neighborhood'] = sandton_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [sandton_venues_onehot.columns[-1]] + list(sandton_venues_onehot.columns[:-1])
sandton_venues_onehot = sandton_venues_onehot[fixed_columns]

sandton_venues_onehot.head()

In [ ]:
sandton_grouped = sandton_venues_onehot.groupby('Neighborhood').mean().reset_index()


In [ ]:
sandton_grouped.groupby('Neighborhood').count()

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = sandton_grouped['Neighborhood']

for ind in np.arange(sandton_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sandton_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

## Step 15 - Run the Clustering Model

In [28]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

sandton_grouped_clustering = sandton_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sandton_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:5] 

NameError: name 'sandton_grouped' is not defined

## Step 16 - Add the Cluster Labels

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

sandton_merged = df_s_xy
print(df_s_xy.head())
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
sandton_merged = sandton_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Name')

In [ ]:
sandton_merged # check the last columns!

## Cluster 0

In [ ]:
sandton_merged.loc[sandton_merged['Cluster Labels'] == 0, sandton_merged.columns[[1] + list(range(5, sandton_merged.shape[1]))]]

## Cluster 1

In [ ]:
sandton_merged.loc[sandton_merged['Cluster Labels'] == 1, sandton_merged.columns[[1] + list(range(5, sandton_merged.shape[1]))]]

## Cluster 2

In [ ]:
sandton_merged.loc[sandton_merged['Cluster Labels'] == 2, sandton_merged.columns[[1] + list(range(5, sandton_merged.shape[1]))]]

In [ ]:
sandton_merged.loc[sandton_merged['Cluster Labels'] == 3, sandton_merged.columns[[1] + list(range(5, sandton_merged.shape[1]))]]

In [ ]:
sandton_merged.loc[sandton_merged['Cluster Labels'] == 4, sandton_merged.columns[[1] + list(range(5, sandton_merged.shape[1]))]]

In [ ]:
# create map of Sandton using latitude and longitude values
map_sandton_cluster = folium.Map(location=[-26.064634, 28.034603], zoom_start=12)
from folium.plugins import HeatMap

sandton_merged['marker_color'] = pd.cut(sandton_merged['Cluster Labels'], bins=5, 
                              labels=['blue', 'yellow', 'red','orange','green'])

# add markers to map
for lat, lng, Name, Population, cluster, color in zip(sandton_merged['Latitude'], sandton_merged['Longitude'], sandton_merged['Name'], sandton_merged['Population'],sandton_merged['Cluster Labels'],sandton_merged['marker_color']):
    label = '{}, {}, {}'.format(Name, Population,cluster)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lng)],
        radius=7,
        popup=label,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        parse_html=False).add_to(map_sandton_cluster)  
        
map_sandton_cluster